In [2]:
from collections import defaultdict
from itertools import chain
import json
from pathlib import Path

import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from nltk import word_tokenize
import numpy as np

Using TensorFlow backend.


In [3]:
data_path = Path('/home/ilai/CMU/Fall2017/11777_Advanced_Multimodal_Machine_Learning/Project/visual-concepts/preprocessing')
visual_concepts_path = Path('/home/ilai/CMU/Fall2017/11777_Advanced_Multimodal_Machine_Learning/Project/visual-concepts/data')
annotations_path = visual_concepts_path.joinpath('annotations')

In [4]:
# punctuations to be removed from the sentences, from scripts/script_1.py
punctuations = ["''", "'", "``", "`", "-LRB-", "-RRB-", "-LCB-", "-RCB-",
  ".", "?", "!", ",", ":", "-", "--", "...", ";"] 

def tokenize_caption(caption):
    return [token.lower() for token in word_tokenize(caption) if token not in punctuations]

def load_captions(annotations_path, dataset):
    annotations_json = annotations_path.joinpath('captions_{}2014.json'.format(dataset)).read_text()
    annotations = json.loads(annotations_json)
    captions = defaultdict(lambda: [])
    for c in annotations['annotations']:
        captions[c['image_id']].append(tokenize_caption(c['caption']))
    return captions

def load_batch(batches_path, captions_dict, batch_id):
    batch_path = batches_path.joinpath('batch{}.npz'.format(batch_id))
    npzfile = np.load(batch_path)
    ids = sorted(int(key) for key in npzfile.keys())
    data_batch = [npzfile[str(id_)] for id_ in ids]
    npzfile.close()
    captions_batch = [captions_dict[id_] for id_ in ids]
    return data_batch, captions_batch

def load_data(data_path, dataset):
    batches_path = data_path.joinpath(dataset)
    captions_dict = load_captions(annotations_path, dataset)
    num_batches = sum(1 for _ in batches_path.glob('*.npz'))
    data_batches, captions_batches = zip(*[load_batch(batches_path, captions_dict, batch_id)
                                           for batch_id in range(num_batches)])
    data = np.asarray(list(chain.from_iterable((data_batches))))
    captions = list(chain.from_iterable(captions_batches))
    return data, captions

In [5]:
def voc(captions_train):
    vocabulary_set = set()
    for tcaptions_lists in captions_train:
        for tcaption in tcaptions_lists:
            vocabulary_set.update(tcaption)
    vocabulary = sorted(vocabulary_set)

    vocabulary += ['<start>', '<end>', '<UNK>']
    word_to_ix = {word: i for i, word in enumerate(vocabulary)}
    return vocabulary, word_to_ix

def encode_captions(captions, word_to_ix):
    unknown_ix = word_to_ix['<UNK>']
    encoded_captions = np.ones((data_train.shape[0], num_captions_per_example, max_len_caption), dtype=int) * word_to_ix['<end>']
    encoded_captions[:, :, 0] = word_to_ix['<start>']
    caption_lengths = np.zeros((data_train.shape[0], num_captions_per_example), dtype=int)
    for i, tcaption_list in enumerate(captions):
        for j in range(num_captions_per_example):
            tcaption = tcaption_list[j]
            caption_lengths[i, j] = len(tcaption) + 2 # num words + attrs + <start>
            for k, word in enumerate(tcaption):
                encoded_captions[i, j, k+1] = word_to_ix.get(word, unknown_ix)
    return encoded_captions, caption_lengths

In [214]:
enc_captions_train

array([[9576, 1725, 5576, ..., 9577, 9577, 9577],
       [9576,   89, 3565, ..., 9577, 9577, 9577],
       [9576,   89, 3252, ..., 9577, 9577, 9577],
       ..., 
       [9576, 1623, 1118, ..., 9577, 9577, 9577],
       [9576,   89, 4609, ...,  780, 4345, 9577],
       [9576,   89, 4490, ..., 9577, 9577, 9577]])

In [6]:
num_captions_per_example = 5
max_len_caption = 60

data_train, captions_train = load_data(data_path, 'train')

vocabulary, word_to_ix = voc(captions_train)
enc_captions_train, caption_lengths_train = encode_captions(captions_train, word_to_ix)

# Keep one caption per example for now
enc_captions_train = enc_captions_train[:, 0, :]
caption_lengths_train = caption_lengths_train[:, 0]

max_size_caption = 15
if max_size_caption is not None:
    max_len_caption = max_size_caption
    captions_to_keep = caption_lengths_train <= max_size_caption
    enc_captions_train = enc_captions_train[captions_to_keep][:, :max_size_caption]
    caption_lengths_train = caption_lengths_train[captions_to_keep]
    data_train = data_train[captions_to_keep]

In [204]:
enc_captions_train.shape

(8475, 15)

In [157]:
enc_captions_train.shape

(9175, 60)

In [10]:
embedding_dim = 10
voc_dim = len(vocabulary)

In [7]:
new_attr_size = 100
if new_attr_size is not None:
    data_train = data_train[:, :new_attr_size]

In [8]:
data_train.shape

(8475, 100)

In [11]:
tf.reset_default_graph()

attrs_dim = data_train.shape[1]
attrs = tf.placeholder(tf.float32, [None, attrs_dim])
Ta = tf.get_variable('Ta',
                     shape=[attrs_dim, embedding_dim],
                     initializer=tf.random_normal_initializer())
xm1 = tf.matmul(attrs, Ta) # x^{-1}

captions = tf.placeholder(tf.int32, [None, max_len_caption])
caption_1_hot = tf.one_hot(indices=captions,
                           depth=voc_dim,
                           axis=-1)
Ts = tf.get_variable('Ts',
                     shape=[voc_dim, embedding_dim],
                     initializer=tf.random_normal_initializer())

c1hr = tf.reshape(caption_1_hot, [-1, voc_dim])
xr = tf.matmul(c1hr, Ts)
x = tf.reshape(xr, [-1, max_len_caption, embedding_dim])
xs = tf.split(x, max_len_caption, axis=1)
xs = [tf.reshape(xsi, [-1, embedding_dim]) for xsi in xs]

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=voc_dim)

captions_l = tf.placeholder(tf.int32, [None])
outputs, _ = tf.nn.static_rnn(lstm_cell,
                              [xm1] + xs[:-1],
                              dtype=tf.float32,
                              sequence_length=captions_l)

costs_per_output = [tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=captions[:, i+1]))
                    for i, out in enumerate(outputs[1:])]
cost = tf.add_n(costs_per_output) / tf.cast(captions_l, tf.float32)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [ ]:
batch_size=1

sess = tf.Session(config=tf.ConfigProto(
        device_count = {'GPU': 0}
    )
)
init = tf.global_variables_initializer()
sess.run(init)
_, out = sess.run([train, cost],
               feed_dict={captions: enc_captions_train[:batch_size],
                          captions_l: caption_lengths_train[:batch_size],
                          attrs: data_train[:batch_size]})

In [ ]:
out